# 🔍 Exploration des données - Annonces immobilières Notaires

**Objectif** : Explorer et documenter les données brutes pour préparer le script `clean_data.py`

**Source** : Scraping de https://www.immobilier.notaires.fr/

**Note importante** : Ce notebook est un outil d'exploration. Il ne produit aucun fichier.  
Les transformations identifiées seront implémentées dans `src/clean_data.py`.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Configuration
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Chargement des données brutes
df = pd.read_csv("../data/annonces_raw.csv")
print(f"{len(df)} annonces chargées")

1292 annonces chargées


---
## 1. Aperçu général des données

In [3]:
df.head(3)

,Departement,URL,Titre,Type_Bien,Localisation,Prix,Surface_m2,Nb_Pieces,Description
0,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T4- Paris 8 - Paris (75),Appartement T4,- Paris 8 - Paris (75),955000,72.12,4,Notaire - F4 Rue Berryer PARIS VIIIème. Au coe...
1,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T2- Paris 16 - Paris (75),Appartement T2,- Paris 16 - Paris (75),745500,70.00,2,Notaire vend un appartement T2 de 70.58m² Carr...
2,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T4- Paris 15 - Paris (75),Appartement T4,- Paris 15 - Paris (75),600000,74.15,4,"Exclusivité, au sein d'un bel immeuble d'angle..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Departement   1292 non-null   int64  
 1   URL           1292 non-null   object 
 2   Titre         1292 non-null   object 
 3   Type_Bien     1292 non-null   object 
 4   Localisation  1292 non-null   object 
 5   Prix          1292 non-null   int64  
 6   Surface_m2    1142 non-null   float64
 7   Nb_Pieces     1292 non-null   int64  
 8   Description   1292 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 91.0+ KB


In [5]:
df.describe()

,Departement,Prix,Surface_m2,Nb_Pieces
count,1292.000000,1.292000e+03,1142.000000,1292.000000
mean,43.382353,3.462838e+05,115.702898,3.493034
std,22.573112,3.957543e+05,70.421674,1.697708
min,1.000000,1.900000e+04,6.550000,0.000000
25%,30.000000,1.562825e+05,72.000000,2.000000
50%,42.000000,2.499000e+05,102.880000,3.000000
75%,62.000000,3.974000e+05,144.000000,4.000000
max,83.000000,4.935000e+06,858.000000,22.000000


### 📋 Observations
- **1292 annonces** sur 22 départements
- **9 colonnes** : Departement, URL, Titre, Type_Bien, Localisation, Prix, Surface_m2, Nb_Pieces, Description
- Types de données corrects (int64 pour les numériques, object pour le texte)

---
## 2. Qualité des données

### 2.1 Doublons

In [6]:
print(f"Doublons complets : {df.duplicated().sum()}")
print(f"Doublons sur URL : {df['URL'].duplicated().sum()}")

Doublons complets : 0
Doublons sur URL : 0


**Aucun doublon** - Le scraper a bien fonctionné

### 2.2 Valeurs manquantes

In [7]:
missing = df.isna().sum()
missing_pct = (missing / len(df) * 100).round(1)

pd.DataFrame({
    'Manquantes': missing,
    'Pourcentage': missing_pct
})

,Manquantes,Pourcentage
Departement,0,0.0
URL,0,0.0
Titre,0,0.0
Type_Bien,0,0.0
Localisation,0,0.0
Prix,0,0.0
Surface_m2,150,11.6
Nb_Pieces,0,0.0
Description,0,0.0


**150 valeurs manquantes sur Surface_m2** (11.6%)

→ À traiter dans `clean_data.py`

---
## 3. Traitement des surfaces manquantes

Hypothèse : La surface peut être extraite de la colonne `Description` via regex

### 3.1 Test du pattern regex principal

In [8]:
# Extraction de la surface depuis le texte à l'aide d'une expression régulière

pattern_m2 = r"(\d+(?:[.,]\d+)?)\s*m²"


missing_surface = df[df['Surface_m2'].isna()]


for i, row in missing_surface.head(3).iterrows():
    desc = row['Description'][:200]
    match = re.search(pattern_m2, desc)
    print(f"Description: {desc}...")
    print(f"Surface extraite : {match.group(1) if match else 'NON'}\n")

Description: APPARTEMENT A VENDRE A PARIS 15EME EN EXCLUSIVITE ! N'attendez plus pour découvrir cet agréable appartement de type 3 d'environ 67 m2, au troisième étage avec ascenseur d'un immeuble datant de 1976 et...
Surface extraite : NON

Description: Vendu LOUÉ. Paris 20ème, Gambetta, dans un immeuble ancien à deux pas de la Place, appartement de 33,6m² au 5ème étage avec ascenseur. Plan en étoile pour cet appartement vendu loué 987 +105 de char...
Surface extraite : 33,6

Description: À VENDRE : Appartement idéalement situé dans le 15ème arrondissement de Paris, à 5 minutes de la Médiathèque Marguerite-Yourcenar, à 4 minutes du métro Vaugirard, et à seulement 22 minutes à pied de l...
Surface extraite : NON



### 3.2 Patterns alternatifs identifiés

Certaines annonces utilisent des formats différents :

In [9]:
# Test de patterns alternatifs
patterns_test = [
    (r"(\d+(?:[.,]\d+)?)\s*m²", "m² (standard)"),
    (r"(\d+(?:[.,]\d+)?)\s*m2", "m2 (sans accent)"),
    (r"environ\s*(\d+)", "'environ X'"),
]

for pattern, name in patterns_test:
    count = missing_surface['Description'].str.contains(pattern, regex=True, na=False).sum()
    print(f"{name}: {count} matches")

m² (standard): 93 matches
m2 (sans accent): 17 matches
'environ X': 34 matches


C:\Users\hello\AppData\Local\Temp\ipykernel_20864\2103631227.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  count = missing_surface['Description'].str.contains(pattern, regex=True, na=False).sum()


Lors de l'analyse des descriptions d'annonces, nous avons constaté que la surface n'est pas toujours écrite de la même manière. En effet, selon les annonceurs, on retrouve différentes conventions d'écriture :
Le format le plus courant utilise le symbole "m²" avec l'exposant (par exemple "75 m²"). Ce pattern standard nous permet de récupérer 93 descriptions parmi celles où la surface est manquante.
Cependant, certains rédacteurs utilisent "m2" sans accent, probablement par facilité de saisie au clavier. Ce format alternatif concerne 17 descriptions supplémentaires.
Enfin, une formulation plus littéraire apparaît dans certaines annonces avec l'expression "environ" suivie d'un nombre (par exemple "environ 80 mètres carrés"). Ce cas de figure représente 34 occurrences.
Cette analyse préliminaire nous permet de définir une stratégie d'extraction multi-patterns : plutôt que de chercher un seul format, notre script clean_data.py testera successivement ces trois patterns pour maximiser le taux de récupération des surfaces manquantes.

### 3.3 Résultat de l'extraction multi-patterns

In [10]:
def extract_surface_smart(description, type_bien=None):
    """
    Extraction INTELLIGENTE de la surface.
    
    ATTENTION : Les descriptions mentionnent souvent plusieurs surfaces :
    - Surface de l'appartement (ce qu'on veut)
    - Surface du balcon (6-15 m²)
    - Surface de la cave (4-10 m²)
    - Surface de la terrasse (10-30 m²)
    
    Stratégie d'extraction (par ordre de priorité) :
    1. Surface Carrez (mention légale, la plus fiable)
    2. Surface 'habitable'
    3. Formulation 'de X m²' ou 'd'environ X m²'
    4. Plus grande surface au-dessus d'un seuil minimum
    
    Le seuil minimum est adapté au type de bien :
    - T0/T1 (studios) : 12 m² minimum
    - T2+ : 25 m² minimum
    """
    desc = str(description)
    pattern = r"(\d+(?:[.,]\d+)?)\s*m[²2]"
    
    # Seuil adapté au type de bien
    if type_bien and ('T0' in str(type_bien) or 'T1' in str(type_bien)):
        seuil_min = 12
    else:
        seuil_min = 25
    
    # Priorité 1 : Surface Carrez
    carrez = re.search(r"(\d+(?:[.,]\d+)?)\s*m[²2]\s*(?:carrez|loi carrez)", desc, re.IGNORECASE)
    if carrez:
        return float(carrez.group(1).replace(',', '.'))
    
    # Priorité 2 : Surface habitable
    habitable = re.search(r"(\d+(?:[.,]\d+)?)\s*m[²2]\s*habitables?", desc, re.IGNORECASE)
    if habitable:
        return float(habitable.group(1).replace(',', '.'))
    
    # Priorité 3 : Formulation standard
    formulation = re.search(r"(?:de|d'environ|d'une surface de)\s*(\d+(?:[.,]\d+)?)\s*m[²2]", desc, re.IGNORECASE)
    if formulation:
        val = float(formulation.group(1).replace(',', '.'))
        if val >= seuil_min:
            return val
    
    # Priorité 4 : Plus grande surface > seuil
    matches = re.findall(pattern, desc, re.IGNORECASE)
    if matches:
        values = [float(m.replace(',', '.')) for m in matches]
        valid = [v for v in values if v >= seuil_min]
        if valid:
            return max(valid)
        return max(values)
    
    return None

# Test sur cas problématiques
test_cases = [
    ('Appartement T3', 'balcon de 9 m², appartement de 63 m² avec cave de 4 m²'),
    ('Appartement T1', 'studio de 18 m² avec balcon de 3 m²'),
    ('Appartement T4', '85 m² Carrez avec terrasse 15 m²'),
]

print("Test de l'extraction intelligente :\n")
for type_bien, desc in test_cases:
    result = extract_surface_smart(desc, type_bien)
    print(f"{type_bien}: {result} m² ← '{desc[:50]}...'")

Test de l'extraction intelligente :

Appartement T3: 63.0 m² ← 'balcon de 9 m², appartement de 63 m² avec cave de ...'
Appartement T1: 18.0 m² ← 'studio de 18 m² avec balcon de 3 m²...'
Appartement T4: 85.0 m² ← '85 m² Carrez avec terrasse 15 m²...'


In [11]:
# Application sur les données manquantes
missing_surface['surface_smart'] = missing_surface.apply(
    lambda row: extract_surface_smart(row['Description'], row['Type_Bien']), 
    axis=1
)

recovered = missing_surface['surface_smart'].notna().sum()
still_missing = len(missing_surface) - recovered

print(f"Sur 150 surfaces manquantes :")
print(f"  ✅ Récupérables (extraction intelligente) : {recovered}")
print(f"  ❌ Irrécupérables : {still_missing}")
print(f"  📊 Taux de récupération : {recovered/150*100:.1f}%")

Sur 150 surfaces manquantes :
  ✅ Récupérables (extraction intelligente) : 110
  ❌ Irrécupérables : 40
  📊 Taux de récupération : 73.3%


C:\Users\hello\AppData\Local\Temp\ipykernel_20864\1190401907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_surface['surface_smart'] = missing_surface.apply(


L'extraction intelligente des surfaces applique une stratégie à plusieurs niveaux pour éviter les erreurs. Plutôt que de prendre la première surface trouvée dans la description (qui pourrait être celle d'un balcon ou d'une cave), la fonction cherche d'abord les mentions légales fiables comme la surface Carrez, puis les formulations explicites comme "appartement de X m²". En dernier recours, elle sélectionne la plus grande surface qui dépasse un seuil minimum adapté au type de bien. Cette approche permet de récupérer correctement 110 surfaces sur les 150 manquantes, soit un taux de récupération de 73.3%.

### ⚠️ Point d'attention documenté

**Problème identifié** : Certaines descriptions mentionnent plusieurs surfaces (balcon, cave, terrasse).  
Une extraction naïve (première valeur trouvée) donnerait des résultats erronés.

**Exemple** : "balcon de 9 m², appartement de 63 m²"  
- ❌ Extraction naïve → 9 m² (FAUX)
- ✅ Extraction intelligente → 63 m² (CORRECT)

**Solution implémentée dans `clean_data.py`** :
1. Chercher d'abord la surface Carrez (fiable légalement)
2. Sinon, chercher la formulation "de X m²"
3. En dernier recours, prendre la plus grande surface > seuil minimum
4. Adapter le seuil au type de bien (12m² pour studios, 25m² pour T2+)

### 3.4 Analyse des cas irrécupérables

In [12]:
# Identification des cas restants
# On utilise la colonne déjà calculée 'surface_smart'
irrecuperables = missing_surface[missing_surface['surface_smart'].isna()]

print(f"Répartition des {len(irrecuperables)} cas irrécupérables :\n")
print("Par département :")
print(irrecuperables['Departement'].value_counts().head(10))

Répartition des 40 cas irrécupérables :

Par département :
Departement
74    8
33    4
35    4
51    4
76    3
6     3
59    2
83    2
29    2
21    2
Name: count, dtype: int64


In [13]:
print("Par type de bien :")
print(irrecuperables['Type_Bien'].value_counts())

Par type de bien :
Type_Bien
Appartement T2    15
Appartement T3    12
Appartement T4     7
Appartement T5     4
Appartement T1     2
Name: count, dtype: int64


### 📋 Décision pour les ~40 cas irrécupérables

**Option retenue : SUPPRESSION**

Justification :
- 40 lignes = 3.1% des données → impact minimal
- Sans surface, impossible de calculer le prix/m² (variable clé de l'analyse)
- L'imputation par médiane biaiserait les résultats
- Données restantes (1252 lignes) = largement suffisant pour l'analyse

→ **À implémenter dans `clean_data.py`** : suppression des lignes sans surface après extraction regex

---
## 4. Analyse de la colonne Localisation

Objectif : Extraire le nom de la ville pour le géocodage

In [14]:
# Exemples de localisations
df['Localisation'].head(40)

0      - Paris 8 - Paris (75)
1     - Paris 16 - Paris (75)
2     - Paris 15 - Paris (75)
3     - Paris 14 - Paris (75)
4     - Paris 16 - Paris (75)
5      - Paris 5 - Paris (75)
6     - Paris 15 - Paris (75)
7     - Paris 12 - Paris (75)
8      - Paris 4 - Paris (75)
9     - Paris 16 - Paris (75)
10     - Paris 6 - Paris (75)
11    - Paris 14 - Paris (75)
12    - Paris 15 - Paris (75)
13    - Paris 13 - Paris (75)
14    - Paris 15 - Paris (75)
15     - Paris 7 - Paris (75)
16    - Paris 14 - Paris (75)
17    - Paris 15 - Paris (75)
18    - Paris 14 - Paris (75)
19    - Paris 20 - Paris (75)
20    - Paris 15 - Paris (75)
21    - Paris 12 - Paris (75)
22    - Paris 18 - Paris (75)
23     - Paris 9 - Paris (75)
24     - Paris 5 - Paris (75)
25    - Paris 18 - Paris (75)
26    - Paris 15 - Paris (75)
27     - Paris 8 - Paris (75)
28     - Paris 1 - Paris (75)
29    - Paris 20 - Paris (75)
30     - Paris 9 - Paris (75)
31    - Paris 11 - Paris (75)
32     - Paris 4 - Paris (75)
33     - P

In [15]:
# Pattern observé : "- Ville - Département (XX)"
# Exemple : "- Paris 8 - Paris (75)"

def extraire_ville(localisation):
    """Extrait le nom de la ville depuis la localisation"""
    # Retirer le tiret initial
    texte = localisation.lstrip('- ').strip()
    # Prendre la partie avant le second tiret
    if ' - ' in texte:
        return texte.split(' - ')[0].strip()
    return texte

# Test
print("Test d'extraction de ville :\n")
for loc in df['Localisation'].head(5):
    print(f"{loc} → {extraire_ville(loc)}")

Test d'extraction de ville :

- Paris 8 - Paris (75) → Paris 8
- Paris 16 - Paris (75) → Paris 16
- Paris 15 - Paris (75) → Paris 15
- Paris 14 - Paris (75) → Paris 14
- Paris 16 - Paris (75) → Paris 16


In [16]:
# Nombre de villes uniques
villes = df['Localisation'].apply(extraire_ville)
print(f"Nombre de villes uniques : {villes.nunique()}")
print(f"\nTop 10 des villes :")
print(villes.value_counts().head(10))

Nombre de villes uniques : 674

Top 10 des villes :
Localisation
Reims          31
Grenoble       29
Dijon          22
St Etienne     18
Montpellier    17
Angers         15
Uzes           14
Bordeaux       14
Nantes         12
Paris 15       11
Name: count, dtype: int64


### 📋 Note pour le géocodage

Les villes Paris/Lyon/Marseille sont avec arrondissement (ex: "Paris 8", "Lyon 3").  
→ Le script `clean_data.py` devra gérer ces cas spéciaux pour Nominatim (conversion en code postal)

---
## 5. Résumé des transformations à implémenter

### Fichier `clean_data.py` devra :

1. **Charger** `data/annonces_raw.csv`

2. **Extraire les surfaces manquantes** avec extraction INTELLIGENTE :
   - Priorité 1 : Surface Carrez (la plus fiable)
   - Priorité 2 : Surface habitable
   - Priorité 3 : Formulation "de X m²"
   - Priorité 4 : Plus grande surface > seuil minimum
   - ⚠️ Adapter le seuil au type de bien (12m² T0/T1, 25m² T2+)

3. **Supprimer les lignes** où Surface_m2 reste NaN (~40 lignes, 3.1%)

4. **Créer la colonne Ville** à partir de Localisation

5. **Créer la colonne prix_m2** = Prix / Surface_m2

6. **Géocoder les villes** (latitude, longitude) avec Nominatim
   - Gérer les arrondissements Paris/Lyon/Marseille (conversion en CP)
   - Utiliser un cache pour optimiser

7. **Sauvegarder** `data/annonces_clean.csv`

### Colonnes finales attendues :
```
Departement, URL, Titre, Type_Bien, Localisation, Prix, Surface_m2, 
Nb_Pieces, Description, Ville, prix_m2, latitude, longitude
```

---
*Notebook d'exploration - Ne produit aucun fichier*  
*Les transformations sont implémentées dans `src/clean_data.py`*

---
# PARTIE 2 : ANALYSES SUR DONNÉES NETTOYÉES
---

Après exécution de `clean_data.py`, nous disposons du fichier `annonces_clean.csv`.  
Cette partie présente les analyses statistiques et visualisations sur les données propres.

## 6. Analyses et visualisations

In [17]:
# Chargement des données NETTOYÉES
df_clean = pd.read_csv("../data/annonces_clean.csv")

print(f"✅ {len(df_clean)} annonces nettoyées chargées")
print(f"Colonnes : {list(df_clean.columns)}")

✅ 1252 annonces nettoyées chargées
Colonnes : ['Departement', 'URL', 'Titre', 'Type_Bien', 'Localisation', 'Prix', 'Surface_m2', 'Nb_Pieces', 'Description', 'Ville', 'Nom_Departement', 'prix_m2', 'latitude', 'longitude']


In [18]:
# Aperçu des données nettoyées
df_clean.head()

,Departement,URL,Titre,Type_Bien,Localisation,Prix,Surface_m2,Nb_Pieces,Description,Ville,Nom_Departement,prix_m2,latitude,longitude
0,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T4- Paris 8 - Paris (75),Appartement T4,- Paris 8 - Paris (75),955000,72.12,4,Notaire - F4 Rue Berryer PARIS VIIIème. Au coe...,Paris 8,Paris,13241.82,48.873379,2.311153
1,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T2- Paris 16 - Paris (75),Appartement T2,- Paris 16 - Paris (75),745500,70.00,2,Notaire vend un appartement T2 de 70.58m² Carr...,Paris 16,Paris,10650.00,48.854928,2.255330
2,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T4- Paris 15 - Paris (75),Appartement T4,- Paris 15 - Paris (75),600000,74.15,4,"Exclusivité, au sein d'un bel immeuble d'angle...",Paris 15,Paris,8091.71,48.841430,2.296165
3,75,https://www.immobilier.notaires.fr/fr/annonce-...,Vente en viager :Appartement T3- Paris 14 - Pa...,Appartement T3,- Paris 14 - Paris (75),995000,87.59,3,VENTE DE LA NUE-PROPRIÉTÉ\nPrix : 995 000 € FA...,Paris 14,Paris,11359.74,48.829558,2.323974
4,75,https://www.immobilier.notaires.fr/fr/annonce-...,Achat :Appartement T8- Paris 16 - Paris (75),Appartement T8,- Paris 16 - Paris (75),2961150,245.96,8,Paris 16e – La Muette / Passy\nAppartement de ...,Paris 16,Paris,12039.15,48.854928,2.255330


In [19]:
# Vérification : plus de valeurs manquantes sur les colonnes clés
print("Valeurs manquantes :")
print(df_clean[['Prix', 'Surface_m2', 'prix_m2', 'latitude', 'longitude']].isna().sum())

Valeurs manquantes :
Prix          0
Surface_m2    0
prix_m2       0
latitude      0
longitude     0
dtype: int64


---
*Notebook d'exploration*  
*Données nettoyées par `src/clean_data.py` → Visualisation interactive dans `src/dashboard.py`*